In [1]:
import os
import sys 
import numpy as np 
import tensorflow as tf 
import time 

c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:522: Futu

In [9]:
def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path) 
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
  
def load_validation_data(rootpath, model=1):
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...") 
    if model ==1: # SP process
        testX  =np.load(os.path.join(rootpath, 'testX_pred.npy') ) 
        testY  =np.load(os.path.join(rootpath, 'testY_pred.npy')) 
        Energy =np.load(os.path.join(rootpath, 'Energy_pred.npy')) 
    elif model==2: # NS process
        testX  =np.load(os.path.join(rootpath, 'testX_no.npy') ) 
        testY  =np.load(os.path.join(rootpath, 'testY_no.npy')) 
        Energy =np.load(os.path.join(rootpath, 'Energy_no.npy'))  
    batch_size =np.shape(testX)[0] 
    return (testX,testY,Energy)

def validate_model(session, val_data, x,y, e, phase_train,prob ): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...") 
    predict_op = tf.argmax(prob, 1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y, val_e = val_data
    test_correct = session.run(correct,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    test_acc = session.run(acc,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True})
    return ( test_acc), test_correct  

def extract_validation_handles(session):
    """ Extracts the input and predict_op handles that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholder  """ 
    valid_nodes = tf.get_collection_ref("validation_nodes") 
    x = valid_nodes[0] 
    y = valid_nodes[1]
    e = valid_nodes[2]  
    phase_train =valid_nodes[3]
    prob = valid_nodes[4]
    global batch_size  
    return (x,y,e, phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1)
    acc = []
    
    zero = np.where(label_y==0)
    correctl=[correct_results[i] for i in zero]
    accuracy_zero=1-np.mean(correctl)
    acc.append(accuracy_zero)
    
    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one]
    accuracy_one=1-np.mean(correctg)
    acc.append(accuracy_one)
    
    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two]
    accuracy_two=1-np.mean(correctt) 
    acc.append(accuracy_two)
    
    three = np.where(label_y==3)
    correctv=[correct_results[i] for i in three]
    accuracy_three=1-np.mean(correctv)
    acc.append(accuracy_three)
   
    #print('%.1f'%(100-100*accuracy_zero))
    #print('%.1f'%(100-100*accuracy_one))
    #print('%.1f'%(100-100*accuracy_two))
    #print(100-100*accuracy_three)  
    return acc, correctl,correctg,correctt 


def reset():
    tf.reset_default_graph() 
    
batch_size =50  
model =1# model = 1: SP process; model = 2: NS process 
test_name = 'one_sec' # choose from{  total,one_sec,two_sec, half_sec, one_half_sec } 
rootpath =os.path.join('/data/test/sub_features', test_name)
save_path = '/codes/saved_models/ModelA/modelA_ratio100_set01'  
with tf.Session() as session:   
    x,y,e,phase_train,prob = load_model(session, save_path)
    val_data = load_validation_data(rootpath, model = model)  
    accuracy ,correct_results= validate_model(session, val_data, x,y,e,phase_train,prob) 
    acc,correct_l,correct_g,correct_t= each_perform(correct_results,val_data[1] ) 
    #reset() 
    session.close()     
id_True =  np.nonzero(correct_results==True) 
overall_acc = (len(id_True[0])/len(correct_results))

print ('LT:', 100*np.mean( correct_l))
print ('GT:', 100*np.mean( correct_g))
print ('TP:', 100*np.mean( correct_t))
print ( '%.1f'%(100*overall_acc)) 

 

Loading model from file 'C:/07_research/14_successive_events/codes/saved_models/ModelA/modelA_ratio100_set01'...
INFO:tensorflow:Restoring parameters from C:/07_research/14_successive_events/codes/saved_models/ModelA/modelA_ratio100_set01
Loading validation data...
Validating model...
LT: 95.87628865979381
GT: 90.0990099009901
TP: 97.2972972972973
94.5


c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
